# A TensorFlow Chatbot

## Agenda
* Seq2seq
* Implementation keys
* Chatbot craze

## Sequence to sequence
### 2つのRNNで構成されている
* エンコーダーが可変長の入力シーケンスに対して固定長のベクトルをマッピングする
* デコーダーは可変長の対象シーケンスを固定長のベクトルへマッピングする
* 2つのRNNは入力シーケンスから与えられた対象シーケンスの条件付き確率を最大化するために共同で訓練される。
=> Vanilla Encorder / Decorder

### TensorFlowにおけるエンコーダーとデコーダー
* Encoderとdecorderはよく異なる重みを持つ
#### 注意事項
* vanilla modelにおいては、それぞれのインプットは固定長のベク取りへエンコードされる、デコーダへパスされるように。
* Attentionメカニズムはデコーダに直接インプットにアクセスすることを可能にする

### Bucketing
* 過剰なパディングによる無意味は演算は避けたい
* 類似のシーケンスグループは伸びて同じバケットになる
* バケット毎の分割されたサブグラフを生成する
* v1.0では以下のように使うことができる
```
tf.contrib.training.bucket_by_sequence_length(max_length, examples, batch_size, bucket_boundaries, capactity=2 * batch_size, dynamic_pad=True)
```
* 演習では、Tensorflowのbucketアルゴリズムを使用してモデルを変換する

### サンプリングされたSoftmax
* 正規化することで計算の複雑度を増えることを避ける
* 少量のサンプリングにて、勾配の負の項を近似する。
* それぞれのステップでは正しい単語wとV中でサンプリングされた単語にのみ関連するベクトルを更新する
* 訓練が終了したら、全対象の語彙を使用して、対象の単語の確率である確率を出力する

### Sampled Softmax
```
if config.NUM_SAMPLES > 0 and config.NUM_SAMPLES < config.DEC_VOCAB:
    weight = tf.get_variable('proj_w', [config.HIDDEN_SIZE, config.DEC_VOCAB])
    bias = tf.get_variable('proj_b', [config.DEC_VOCAB])
    self.output_projection = (w, b)
    
   def samplesd_loss(inputs, labels):
       labesl = tf.reshape(labels, [-1, 1])
       return tf.nn.sampled_softmax_loss(tf.transpose(weight), bias, inputs, labels, config.NUM_SAMPLES, config.DEC_VOCAB)
       
    self.softmax_loss_function = sampled_loss
```

